<a href="https://colab.research.google.com/github/Fenilsavani-and/Portfolio_Optimization/blob/main/Portfolio_Optimization_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [133]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import cvxpy as cp
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression

# Model MVO with 2 different methods 1) regular MVO 2) Tangency Portfolio Method.

**Model 1) MVO Model:**

In [2]:
   from google.colab import drive
   drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# data is already preprocessed with log(n) value:

In [126]:
df = pd.read_csv('/content/drive/My Drive/Portfolio_Optimization/Data_pctg.csv', index_col='Date')


In [127]:
df

,SMCI,DECK,RS,CSL,GDDY,VST,GGG,MANH,LII,WSM,SAIA,LECO,RPM,XPO
Date,,,,,,,,,,,,,,
2019-01-03,0.039735,-0.028044,-0.025039,-0.019115,-0.041229,-0.009383,-0.030799,-0.023702,-0.017952,-0.006578,-0.009901,-0.028995,-0.048544,-0.040243
2019-01-04,-0.008917,-0.031953,0.041320,0.020194,0.039349,0.040595,0.032026,0.034232,0.034676,0.014446,0.019815,0.036438,-0.047741,0.028737
2019-01-07,0.008997,0.027178,-0.002480,0.000000,0.016773,0.016905,0.000481,0.017606,-0.002823,0.026503,-0.003450,0.004569,0.010524,0.039705
2019-01-08,-0.006369,0.007354,0.037845,0.015340,-0.008798,0.000426,0.017793,0.031142,0.013425,0.026975,0.026057,0.020720,0.035789,0.017911
2019-01-09,-0.004487,0.004761,0.004525,0.013842,0.006499,-0.006817,0.009213,0.011857,0.012481,0.004503,0.014740,0.013492,0.010603,0.029880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,-0.047538,-0.014827,0.013154,0.004474,-0.009261,0.006086,0.008053,0.001683,0.000631,-0.005812,0.002618,0.008312,0.005643,0.001903
2023-12-26,0.015912,-0.008231,0.000248,0.006826,0.005760,0.004734,0.005442,-0.003996,0.005113,-0.003930,0.013301,0.006778,0.003830,0.006815
2023-12-27,0.002276,-0.000320,0.000566,-0.000891,0.000657,0.004712,0.001152,-0.013132,0.006118,0.003107,-0.013585,0.002820,-0.000621,-0.002219


## **1) MVO model with formula (4.13) equation risky and risk-free assets:**

In [128]:
def mean_variance_opt(mu, cv, Er):
    n = len(mu)
    I = np.ones((n, 1))
    A = np.dot(np.dot(I.T, np.linalg.inv(cv)), mu)       # A,B,C,D > 0
    B = np.dot(np.dot(mu.T, np.linalg.inv(cv)), mu)
    C = np.dot(np.dot(I.T, np.linalg.inv(cv)), I)
    D = B * C - A**2
    lam = (C * Er - A) / D
    gam = (B - A * Er) / D
    wts = lam[0] * np.dot(np.linalg.inv(cv), mu) + gam[0] * np.dot(np.linalg.inv(cv), I) # First Equation.
    g = (B * np.dot(np.linalg.inv(cv), I) - A * np.dot(np.linalg.inv(cv), mu)) / D
    h = (C * np.dot(np.linalg.inv(cv), mu) - A * np.dot(np.linalg.inv(cv), I)) / D
    wts = g + h * Er   #Second Equation.
    return wts


# Example usage:
mu = np.round(df.mean().values.flatten()* 100, 4).reshape(-1, 1)   # Mean Return of last 5 years. ans is ex: 0.23
# Expected returns of assets
cv = df.cov().to_numpy()  # Covariance matrix
Er = 0.15  # Target expected return

weights = mean_variance_opt(mu, cv, Er)
print("Optimal weights:", weights)



Optimal weights: [[ 0.14001633]
 [ 0.08197483]
 [ 0.178299  ]
 [ 0.10087654]
 [ 0.00916674]
 [ 0.11393335]
 [-0.04380577]
 [ 0.05734808]
 [ 0.08422184]
 [ 0.02527823]
 [ 0.11686465]
 [ 0.18606394]
 [ 0.04472674]
 [-0.09496449]]


In [130]:
std =  np.sqrt(np.dot(np.dot(weights.T, cv),weights))
exp_r = np.dot(weights.T, mu)
print("Standard Deviation of portfolio:",std)
print("Portfolio Expected Returns E(r):", exp_r)

Standard Deviation of portfolio: [[0.01653438]]
Portfolio Expected Returns E(r): [[0.15]]


**1.2) MVO Model with Tangency Portfolio Method:**

In [131]:
def mean_variance_Tangency(mu, cv, Er, rf):
    global A, B, C, D, H, I, wts
    n = len(mu)
    I = np.ones((n, 1))
    A = np.dot(np.dot(I.T, np.linalg.inv(cv)), mu)       # A,B,C,D > 0
    B = np.dot(np.dot(mu.T, np.linalg.inv(cv)), mu)
    C = np.dot(np.dot(I.T, np.linalg.inv(cv)), I)
    D = B * C - A**2
    H = ((B) - 2*(A)*(rf)) + ((C)*(rf**2))


    wts = (1/(A - C*rf))*(np.dot(np.linalg.inv(cv),(mu - rf*I)))
    return wts


# Example usage:
mu = np.round(df.mean().values.flatten()*100, 4).reshape(-1, 1)   # Mean Return of last 5 years. ans is ex: 0.23
# Expected returns of assets
cv = df.cov().to_numpy()  # Covariance matrix
Er = 0.15  # Target expected return
rf = 4 # Risk Free return based on Bond yield 10 year.
MVO_weights = mean_variance_Tangency(mu, cv, Er, rf)
print("Optimal weights:", MVO_weights)



Optimal weights: [[-0.01519032]
 [ 0.0192848 ]
 [ 0.05325958]
 [ 0.06955675]
 [ 0.17379223]
 [ 0.14010349]
 [ 0.14786053]
 [-0.05843659]
 [ 0.20427826]
 [-0.04085716]
 [-0.03356886]
 [ 0.2225845 ]
 [ 0.2257243 ]
 [-0.1083915 ]]


In [132]:
exp_r2= rf + (H/(A-C*rf))
std_2 =  np.sqrt(np.dot(np.dot(weights.T, cv),weights))
print("Portfolio Expected Returns E(r):", exp_r2)
print("Standard Deviation of portfolio:",std_2)

Portfolio Expected Returns E(r): [[0.0650449]]
Standard Deviation of portfolio: [[0.01653438]]


### **Model 2) Treynor Black:**

In [134]:
df2 = pd.read_csv('/content/drive/My Drive/Portfolio_Optimization/datasp.csv', index_col='Date')
df2

,SMCI,DECK,RS,CSL,GDDY,VST,GGG,MANH,LII,WSM,SAIA,LECO,RPM,XPO,SP400,TreasuryBill,ExcessReurn
Date,,,,,,,,,,,,,,,,,
1/3/19,0.038966,-0.028445,-0.025357,-0.019300,-0.042103,-0.009427,-0.031283,-0.023988,-0.018115,-0.006599,-0.009950,-0.029424,-0.049762,-0.041075,-0.015905,-0.004219,-0.011686
1/4/19,-0.008957,-0.032475,0.040489,0.019993,0.038595,0.039793,0.031524,0.033659,0.034088,0.014343,0.019621,0.035790,-0.048918,0.028332,0.032349,0.004237,0.028112
1/7/19,0.008957,0.026815,-0.002483,0.000000,0.016634,0.016764,0.000481,0.017452,-0.002827,0.026158,-0.003456,0.004558,0.010469,0.038937,0.010972,0.016878,-0.005906
1/8/19,-0.006390,0.007327,0.037147,0.015224,-0.008837,0.000426,0.017636,0.030667,0.013336,0.026618,0.025723,0.020508,0.035163,0.017753,0.016919,0.000000,0.016919
1/9/19,-0.004497,0.004750,0.004515,0.013747,0.006478,-0.006841,0.009171,0.011787,0.012404,0.004493,0.014632,0.013402,0.010548,0.029443,0.009124,-0.004149,0.013274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/22/23,-0.048705,-0.014938,0.013068,0.004464,-0.009304,0.006068,0.008021,0.001682,0.000631,-0.005829,0.002615,0.008277,0.005627,0.001901,0.003387,0.003831,-0.000444
12/26/23,0.015787,-0.008265,0.000248,0.006802,0.005743,0.004723,0.005427,-0.004004,0.005100,-0.003938,0.013213,0.006755,0.003823,0.006792,0.007627,0.001908,0.005718
12/27/23,0.002274,-0.000320,0.000566,-0.000892,0.000657,0.004701,0.001151,-0.013218,0.006099,0.003102,-0.013678,0.002816,-0.000621,-0.002222,0.000157,-0.001905,0.002061


In [16]:
list = df2.columns.values[:-3]
list

array(['SMCI', 'DECK', 'RS', 'CSL', 'GDDY', 'VST', 'GGG', 'MANH', 'LII',
       'WSM', 'SAIA', 'LECO', 'RPM', 'XPO'], dtype=object)

Find Beta for stocks with covariance and variance value which aligns regression model:

In [17]:
# testing of beta for SMCI:

# subset_cv = round(df_subset.cov(),7)
# column_names = df_subset.columns
# cv_df_subset = pd.DataFrame(subset_cv, columns=column_names, index=column_names)
# cv_df_subset

# var = round(df_subset['SP400'].var(),7)
# cov = cv_df_subset.iloc[4, 14]
# beta = cov/var
# beta

In [139]:
beta = []
alpha = []

for i in list:

    X = df2['ExcessReurn'].values.reshape(-1,1)  # SP400  in first phase
    y = df2[f'{i}'].values.reshape(-1,1)

    linreg = LinearRegression().fit(X, y)

    beta.append(round(linreg.coef_[0][0],6))
    alpha.append(round(linreg.intercept_[0],6))

print("Beta", beta)
print("\nAlpha", alpha)


Beta [0.004081, -0.005828, -0.003574, 0.004557, -0.001702, 0.003029, -0.003362, -0.000293, 0.002155, -1.2e-05, 0.00486, 0.003612, 0.003454, 0.011098]

Alpha [0.002363, 0.001267, 0.001139, 0.000979, 0.000406, 0.000564, 0.000612, 0.001295, 0.000647, 0.001203, 0.001691, 0.00091, 0.000627, 0.001252]


In [140]:
### fase 2:
df_subset = df2.iloc[:, :-2]
columns_to_keep = ['SMCI', 'DECK', 'RS', 'CSL', 'GDDY', 'VST', 'GGG', 'MANH', 'LII','WSM', 'SAIA', 'LECO', 'RPM', 'XPO','ExcessReurn']

df_subset = df2[columns_to_keep].copy()


In [23]:
df_subset

,SMCI,DECK,RS,CSL,GDDY,VST,GGG,MANH,LII,WSM,SAIA,LECO,RPM,XPO,ExcessReurn
Date,,,,,,,,,,,,,,,
1/3/19,0.038966,-0.028445,-0.025357,-0.019300,-0.042103,-0.009427,-0.031283,-0.023988,-0.018115,-0.006599,-0.009950,-0.029424,-0.049762,-0.041075,-0.011686
1/4/19,-0.008957,-0.032475,0.040489,0.019993,0.038595,0.039793,0.031524,0.033659,0.034088,0.014343,0.019621,0.035790,-0.048918,0.028332,0.028112
1/7/19,0.008957,0.026815,-0.002483,0.000000,0.016634,0.016764,0.000481,0.017452,-0.002827,0.026158,-0.003456,0.004558,0.010469,0.038937,-0.005906
1/8/19,-0.006390,0.007327,0.037147,0.015224,-0.008837,0.000426,0.017636,0.030667,0.013336,0.026618,0.025723,0.020508,0.035163,0.017753,0.016919
1/9/19,-0.004497,0.004750,0.004515,0.013747,0.006478,-0.006841,0.009171,0.011787,0.012404,0.004493,0.014632,0.013402,0.010548,0.029443,0.013274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/22/23,-0.048705,-0.014938,0.013068,0.004464,-0.009304,0.006068,0.008021,0.001682,0.000631,-0.005829,0.002615,0.008277,0.005627,0.001901,-0.000444
12/26/23,0.015787,-0.008265,0.000248,0.006802,0.005743,0.004723,0.005427,-0.004004,0.005100,-0.003938,0.013213,0.006755,0.003823,0.006792,0.005718
12/27/23,0.002274,-0.000320,0.000566,-0.000892,0.000657,0.004701,0.001151,-0.013218,0.006099,0.003102,-0.013678,0.002816,-0.000621,-0.002222,0.002061


In [141]:
# standard deviation value of each stocks and market index:
std = df_subset.std().to_list()
std = [round(s, 6) for s in std]

# Return of stocks and market:
returns = df_subset.mean().to_list()
stk_returns = [round(s, 6) for s in returns]

In [142]:
subset_cv = round(df_subset.cov(),7)
column_names = df_subset.columns
cv_df_subset = pd.DataFrame(subset_cv, columns=column_names, index=column_names)
cv_df_subset



,SMCI,DECK,RS,CSL,GDDY,VST,GGG,MANH,LII,WSM,SAIA,LECO,RPM,XPO,ExcessReurn
SMCI,0.001276,0.000346,0.000255,0.000244,0.000273,0.000180,0.000229,0.000359,0.000223,3.361000e-04,0.000334,0.000234,0.000255,0.000357,2.336000e-04
DECK,0.000346,0.000733,0.000262,0.000244,0.000281,0.000181,0.000240,0.000367,0.000213,3.939000e-04,0.000317,0.000218,0.000227,0.000408,-3.336000e-04
RS,0.000255,0.000262,0.000422,0.000240,0.000184,0.000187,0.000217,0.000246,0.000193,2.708000e-04,0.000285,0.000238,0.000212,0.000322,-2.046000e-04
CSL,0.000244,0.000244,0.000240,0.000413,0.000155,0.000172,0.000214,0.000232,0.000207,2.238000e-04,0.000289,0.000244,0.000214,0.000323,2.608000e-04
GDDY,0.000273,0.000281,0.000184,0.000155,0.000547,0.000184,0.000195,0.000344,0.000152,3.222000e-04,0.000237,0.000158,0.000170,0.000279,-9.740000e-05
VST,0.000180,0.000181,0.000187,0.000172,0.000184,0.000558,0.000157,0.000219,0.000150,1.980000e-04,0.000186,0.000164,0.000148,0.000206,1.734000e-04
GGG,0.000229,0.000240,0.000217,0.000214,0.000195,0.000157,0.000303,0.000252,0.000205,2.494000e-04,0.000273,0.000226,0.000214,0.000289,-1.925000e-04
MANH,0.000359,0.000367,0.000246,0.000232,0.000344,0.000219,0.000252,0.000722,0.000242,3.930000e-04,0.000328,0.000222,0.000247,0.000388,-1.680000e-05
LII,0.000223,0.000213,0.000193,0.000207,0.000152,0.000150,0.000205,0.000242,0.000378,2.707000e-04,0.000266,0.000199,0.000201,0.000290,1.233000e-04
WSM,0.000336,0.000394,0.000271,0.000224,0.000322,0.000198,0.000249,0.000393,0.000271,9.176000e-04,0.000351,0.000244,0.000250,0.000409,-7.000000e-07


In [144]:
# residual of all stocks:
residual_var = []
mkt_cov = cv_df_subset.iloc[14, 14]
for i in range(0,14):
  residual_cv = (cv_df_subset.iloc[i,i]) - ((beta[i]**2)*((mkt_cov)**2))
  residual_var.append(round(residual_cv,7))

print("Residual for all stocks:", residual_var)

Residual for all stocks: [0.001276, 0.0007327, 0.000422, 0.000413, 0.0005474, 0.0005579, 0.0003027, 0.000722, 0.0003779, 0.0009176, 0.0008387, 0.0003211, 0.0003496, 0.0010204]


## stk_return, std, beta, alpha, residual_var

Beta_Mkt = 1, residual_mkt = 0, alpha_mkt = 0.

In [145]:
# step 1) Initial Weight for each assets: this initial weight sum would be 1.

# numerator = alpha[i] / residual_var[i]
denominator = sum(map(lambda i: alpha[i] / residual_var[i], range(0,14)))
Wi = 0
In_wts = []
for i in range(0,14):

  Wi = (alpha[i] / residual_var[i])/denominator
  In_wts.append(round(Wi,6))

print("Initial Weights for assets",In_wts)

Initial Weights for assets [0.073744, 0.068859, 0.107479, 0.094394, 0.029535, 0.040256, 0.08051, 0.071424, 0.068177, 0.052206, 0.080287, 0.112853, 0.071418, 0.048859]


In [146]:
# step 2) Now, we calculate alpha of active portfolio
# Alpha(A) = In_wts[i]*alpha[i]

alpha_A = sum(map(lambda i: In_wts[i]*alpha[i], range(0,14)))
alpha_A

0.001104119964

In [147]:
# step 3) Now, we calculate residual variance of active portfolio:
# residual_A = sum(In_wts[i]**2 * residual_var[i])

residual_A = sum(map(lambda i:((In_wts[i]**2) * residual_var[i]), range(0,14)))
residualA = '{:.10f}'.format(residual_A)
residualA


'0.0000439671'

In [149]:
# step 5) Calculate beta for active portfolio Beta_A:
# beta_A = sum(Int_wts[i]*beta[i])

beta_A = round(sum(map(lambda i:(In_wts[i] * beta[i]), range(0,14))),6)
beta_A

0.001459

In [150]:
# step 4) Initial weight of Active Portfolio when Beta of active portfolio = 1:
# Wo = (alpha_A / residual_A)/(stk_returns[14]/mkt_cov)

Wo = round((alpha_A / residual_A)/(stk_returns[14]/mkt_cov),6)
Wo

-212.835502

In [151]:
# step 6) Final total weight assign to active portfolio:
# WA = Wo / (1+(1-beta_A)*Wo). and passive weight WM = 1 - WA

WA = Wo /(1+ (1-beta_A)*Wo)

print('Weight of Active portfolio', WA)

Weight of Active portfolio 1.0061956135316976


In [152]:
# step 7) Assign weight to all assets after assigning weight to market index:

Wi = [lambda i: (WA * ((alpha[i] / residual_var[i]) / denominator)) for i in range(0, 14)]
Wi_values = [function(i) for i, function in enumerate(Wi)]
TB_weights = [round(value, 6) for value in Wi_values]

TB_weights


[0.0742,
 0.069286,
 0.108144,
 0.094979,
 0.029718,
 0.040506,
 0.081009,
 0.071866,
 0.068599,
 0.05253,
 0.080785,
 0.113552,
 0.07186,
 0.049162]

In [153]:
# step 8) calculate sharpe ratio of entire portfolio:
# SR(m) = R(m)/STD(m)
# SR(p) = sqrt(SR(m)^2 + (alpha_A/residual_A)^2)
from math import sqrt
SRm = stk_returns[14]/(sqrt(mkt_cov))

SRp = sqrt(((SRm)**2) + ((alpha_A/residual_A)**2))
SRp

25.112450098229324

In [154]:
# Risk premium for portfolio:
WM = 1-WA
ERp = (WM + (WA*beta_A))*stk_returns[14] + (WA*alpha_A)
ERp

0.0011428907002740975

In [156]:
VRp = ((WM+WA*beta_A)**2)*mkt_cov + (WA*residual_A)**2
sqrt(VRp)

0.001131953112357387

In [157]:
#Excel method:
# need : betaof complete pf = beta_cp, return of active pf = RA, then we calculate Rp
# calculate Std_Active portfolio = std_A
# calculate std_complete PF = std_CP

WM = 1-WA
RM = stk_returns[14]

beta_cp = (WM*1) + (WA*beta_A)

RA = alpha_A + (beta_A*RM)

RP = WM*RM + WA*RA

std_A = sqrt(((beta_A**2)*mkt_cov) + residual_A )

std_CP = sqrt((beta_cp**2)*(mkt_cov) + (WA*residual_A))

In [158]:
# calculate sharp ratio of active pf and complete pf:

SRA = RA / std_A
print('Active Portfolio Sharpe Ratio:', SRA)

SRP = RP / std_CP
print('Complete Portfolio Sharpe Ratio:', SRP)

Active Portfolio Sharpe Ratio: 0.16480036228961306
Complete Portfolio Sharpe Ratio: 0.16939834859430902


# **Model 3) Black Litterman Theory:**

In [159]:
df2.head(2) # We are using same dataframe from Treynor Black Model:

,SMCI,DECK,RS,CSL,GDDY,VST,GGG,MANH,LII,WSM,SAIA,LECO,RPM,XPO,SP400,TreasuryBill,ExcessReurn
Date,,,,,,,,,,,,,,,,,
1/3/19,0.038966,-0.028445,-0.025357,-0.019300,-0.042103,-0.009427,-0.031283,-0.023988,-0.018115,-0.006599,-0.009950,-0.029424,-0.049762,-0.041075,-0.015905,-0.004219,-0.011686
1/4/19,-0.008957,-0.032475,0.040489,0.019993,0.038595,0.039793,0.031524,0.033659,0.034088,0.014343,0.019621,0.035790,-0.048918,0.028332,0.032349,0.004237,0.028112


In [164]:
# Market Caps:
mcaps = {}
for t in list:
    stock = yf.Ticker(t)
    mcaps[t] = stock.info["marketCap"]

print("Market Caps:\n",mcaps)

Market Caps:
 {'SMCI': 55971336192, 'DECK': 23328800768, 'RS': 18416900096, 'CSL': 17530679296, 'GDDY': 17014722560, 'VST': 21743040512, 'GGG': 15504421888, 'MANH': 15206851584, 'LII': 16818629632, 'WSM': 18299715584, 'SAIA': 14908772352, 'LECO': 14144899072, 'RPM': 14892447744, 'XPO': 14175075328}


In [165]:
pip install PyPortfolioOpt

In [43]:
import pypfopt
from pypfopt import black_litterman, objective_functions
from pypfopt import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier

In [166]:
# Calculating Risk Aversion for Market:
# rsk_A = excess return / market variance

cov_df2 = df2.cov()
sp400_mkt_cov = cov_df2.iloc[14,14]

rsk_av = stk_returns[14] / mkt_cov

# delta = black_litterman.market_implied_risk_aversion(df2['ExcessReurn'])
# print("helo",delta)
# print("helow",rsk_av)

In [167]:
# step 2) Calculate market equilibrium return:
#pi = A.cov.mktwts

#COV of stocks:
cov_stks = cov_df2.iloc[0:14,0:14].values

# mkt_wts:
total_wts = sum(mcaps.values())
mkt_wts=[]
for i in mcaps.keys():
  mkt_wts.append(mcaps[i]/total_wts)
mkt_wts= np.array(mkt_wts)
print("Market Initial weights",mkt_wts)

# PI value: iquillibrium excess return of stocks which is called PI.
PI = rsk_av*(np.dot(cov_stks,mkt_wts.T))
PI

Market Initial weights [0.2013674  0.08392975 0.06625826 0.06306991 0.06121366 0.07822467
 0.05578007 0.05470951 0.06050818 0.06583667 0.05363711 0.05088893
 0.05357838 0.0509975 ]


array([-5.64993163e-05, -3.87712203e-05, -2.97870585e-05, -2.85998594e-05,
       -2.98018827e-05, -2.46911450e-05, -2.71309208e-05, -3.87257116e-05,
       -2.65554733e-05, -4.07883339e-05, -3.97541470e-05, -2.73957452e-05,
       -2.76243745e-05, -4.50643808e-05])

In [168]:
# This is form Model: We will not use this:
'''prior = black_litterman.market_implied_prior_returns(mcaps, rsk_av,cov_stks )
prior '''

'prior = black_litterman.market_implied_prior_returns(mcaps, rsk_av,cov_stks )\nprior '

In [169]:
# Create Views for market:
Q = np.array([0.05, 0.1, 0.4, 0.015, 0.10, 0.15, 0.15, 0.18, 0, 0.15, 0.15, 0.15,0.1, 0.12])

P = np.array([
    [1,0,0,0,0,0,0,0,0,0,0,0,0,0], # SMCI = 0.05
    [0,1,0,0,0,0,0,0,0,0,0,0,0,0], # DECK = 0.10
    [0,0,1,0,0,0,0,0,0,0,0,0,0,-1], # RS >  by 0.15
    [0,0,0,1,0,0,0,0,0,0,0,0,0,0], # CSL = 0.05
    [0,0,0,0,1,0,0,0,0,0,0,0,0,0], # GDDY = 0.10
    [0,0,0,0,0,1,0,0,0,0,0,0,0,0], # VST >  by 0.15
    [0,0,0,0,0,0,1,0,0,0,0,0,0,0], # GGG = 0.05
    [0,0,0,0,0,0,-1,1,0,0,0,0,0,0], # MANH = 0.10
    [0,0,0,0,0,0,0,0,1,0,0,0,0,0], # LII >  by 0.15
    [0,0,0,0,0,0,0,0,0,1,0,0,0,0], # WSM = 0.05
    [0,0,0,0,0,0,0,0,0,0,1,0,0,0], # SAIA = 0.10
    [0,0,0,0,0,0,0,0,0,0,0,1,0,0], # LECO >  by 0.15
    [0,0,0,0,0,0,0,0,0,0,0,0,1,0], # RPM >  by 0.15
    [-1,0,0,0,0,0,0,0,0,0,0,0,0,1] # XPO >  by 0.02

])

# confidences = [0.5, 0.4,0, 0.5, 0, 0.8, 0.5, 0, 0.8, 0.5, 0.4, 0.8, 0.5, 0.5]


In [171]:
# Calculate Omega: taw*P*COV*Pt, where taw = 0.0025

OG = 0.0025*(np.dot(np.dot(P,cov_stks),P.T))
OG

array([[ 3.19024173e-06,  8.65066125e-07, -2.55735013e-07,
         6.10344459e-07,  6.81763336e-07,  4.51290967e-07,
         5.71764911e-07,  3.25903640e-07,  5.56931833e-07,
         8.40348888e-07,  8.35385281e-07,  5.85205289e-07,
         6.37947178e-07, -2.29711335e-06],
       [ 8.65066125e-07,  1.83211297e-06, -3.65340366e-07,
         6.09967654e-07,  7.03225773e-07,  4.51798546e-07,
         5.99983038e-07,  3.17044771e-07,  5.33178232e-07,
         9.84659222e-07,  7.92581803e-07,  5.43974783e-07,
         5.67563934e-07,  1.54485848e-07],
       [-2.55735013e-07, -3.65340366e-07,  1.99576562e-06,
        -2.07823869e-07, -2.39266598e-07, -4.68710234e-08,
        -1.80156687e-07, -1.75079891e-07, -2.43549440e-07,
        -3.45763231e-07, -7.77151850e-07, -2.39355184e-07,
        -3.12216855e-07, -1.49069114e-06],
       [ 6.10344459e-07,  6.09967654e-07, -2.07823869e-07,
         1.03283223e-06,  3.88576658e-07,  4.30122967e-07,
         5.34202537e-07,  4.69215835e-08,  5.

In [172]:
# first term: I will change prior with PI
taw = 0.0025
ft = np.linalg.inv((np.linalg.inv((taw*cov_stks)) + np.dot(P.T,np.dot(np.linalg.inv(OG),P))))

# second term:
st = np.dot(np.linalg.inv(taw*cov_stks),PI) + np.dot(P.T, np.dot(np.linalg.inv(OG),Q))
st

array([ -51020.6712808 , -100279.90225092,  903546.0741753 ,
       -330326.06419303,  -93640.19088615,   -6557.7897824 ,
         67229.80201515,  226727.71077626, -231959.05953056,
        -29124.65308228,  -14009.81503135,  -38087.4575015 ,
        -87852.77254365,   -7063.78628491])

In [173]:
# BL orignal formula: E(mu)
MU = np.dot(ft,st)
MU

array([ 2.49717503e-02,  4.99806144e-02,  2.84985106e-01,  7.48570007e-03,
        4.99850991e-02,  7.49876544e-02,  7.49864345e-02,  1.64980637e-01,
       -1.32777366e-05,  7.49796058e-02,  7.49801229e-02,  7.49863021e-02,
        4.99861878e-02,  8.49774678e-02])

In [174]:
ef = EfficientFrontier(MU, cov_stks)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
BL_weights = [weights[i] for i in weights.keys()]
BL_weights

/usr/local/lib/python3.10/dist-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning:

max_sharpe transforms the optimization problem so additional objectives may not work as expected.



[0.00561,
 0.0353,
 0.31448,
 0.0,
 0.03537,
 0.06511,
 0.06508,
 0.17187,
 0.0,
 0.06496,
 0.06496,
 0.06508,
 0.03539,
 0.07678]

# **Model 4) Sharpe ratio model:**

In [52]:
# SR formula:
#SR = mu - rf/ std deviation


In [175]:
df3 = df2.iloc[:,:-1]
df3.columns


Index(['SMCI', 'DECK', 'RS', 'CSL', 'GDDY', 'VST', 'GGG', 'MANH', 'LII', 'WSM',
       'SAIA', 'LECO', 'RPM', 'XPO', 'SP400', 'TreasuryBill'],
      dtype='object')

In [176]:
rf = df3.std()['TreasuryBill']
denominators = sum(map(lambda i:((stk_returns[i] - rf)/std[i]), range(0,14)))

SRM_weights = []
for i in range(0,14):
  x = ((stk_returns[i] - rf)/std[i])/ denominators
  SRM_weights.append(x)

SRM_weights

[0.046226443345335363,
 0.061263530523999875,
 0.0807837902128498,
 0.08171590451737275,
 0.07114819690914559,
 0.07043904205464424,
 0.0955935521349243,
 0.061721844885993284,
 0.08554896900445046,
 0.05477294776148545,
 0.05717886772855807,
 0.09271144681419398,
 0.08895921531781838,
 0.05193624878922865]

In [177]:
# Maximizing Sharpe ratio
wts = np.array(SRM_weights)
stk_rt = stk_returns[:-1]

In [178]:
stk_rt = np.array(stk_rt)

MAX = (np.dot(wts.T,stk_rt) - (0.000537))/np.sqrt(np.dot(wts.T, np.dot(cov_stks,wts)))
print("Sharpe ratio for SRM Model:",MAX)

Sharpe ratio for SRM Model: 0.02730114838821519


# **Model 4) Most Diversified Portfolio:**

In [179]:
# std deviatio = [n x1] and co_var inverse:
std_stk = std[:-1]

In [180]:
I = np.ones((14,1))
cov_stk = np.linalg.inv(cov_stks)
std_stk = np.array(std_stk)

MDP_weights = np.dot(cov_stk,std_stk)/np.dot(I.T,np.dot(cov_stk,std_stk))
MDP_weights.tolist()

[0.145703251016311,
 0.07582785059668085,
 0.04826689969277528,
 0.10847674486072569,
 0.13920890958789733,
 0.2434642688589877,
 -0.1169150999823512,
 0.013483132406313391,
 0.12655092801506837,
 0.09053565482928277,
 0.09448866516562424,
 -0.06260717189671632,
 0.03884562641759872,
 0.05467034043180226]

In [186]:
# Create Dataframe with all models Weights:
MVO_wts = MVO_weights.tolist()
TB_wts = TB_weights
BL_wts = BL_weights
SRM_wts = SRM_weights
MDP_wts = MDP_weights.tolist()
index = ['SMCI', 'DECK', 'RS', 'CSL', 'GDDY', 'VST', 'GGG', 'MANH', 'LII','WSM', 'SAIA', 'LECO', 'RPM', 'XPO']

df_w = pd.DataFrame({'MVO': MVO_wts,
                     'TB': TB_wts,
                     'BL': BL_wts,
                     'SRM': SRM_wts,
                     'MDP': MDP_wts}, index=index)


In [187]:
df_w['MVO'] = df_w['MVO'].apply(lambda x: x[0]) # This is for convert array value in list

# Add a new row with values
SP400 = [0,-0.006195,0,0,0]  # Values for the new row
ind = "S&P400"
df_w.loc[ind] = SP400


In [188]:
df_w

,MVO,TB,BL,SRM,MDP
SMCI,-0.015190,0.074200,0.00561,0.046226,0.145703
DECK,0.019285,0.069286,0.03530,0.061264,0.075828
RS,0.053260,0.108144,0.31448,0.080784,0.048267
CSL,0.069557,0.094979,0.00000,0.081716,0.108477
GDDY,0.173792,0.029718,0.03537,0.071148,0.139209
VST,0.140103,0.040506,0.06511,0.070439,0.243464
GGG,0.147861,0.081009,0.06508,0.095594,-0.116915
MANH,-0.058437,0.071866,0.17187,0.061722,0.013483
LII,0.204278,0.068599,0.00000,0.085549,0.126551
WSM,-0.040857,0.052530,0.06496,0.054773,0.090536


In [189]:
import plotly.graph_objects as go

fig = go.Figure()

# Add traces for each model
for model, values in df_w.items():
    fig.add_trace(go.Scatter(x=df_w.index, y=values, mode='lines', name=model))

# Customize the layout
fig.update_layout(title='Weights of Stocks for Each Model',
                  xaxis_title='Stock Name',
                  yaxis_title='Weight')

# Show the plot
fig.show()

In [191]:
PF_Value = df_w.mul(1000000).round(2)

PF_Value

,MVO,TB,BL,SRM,MDP
SMCI,-15190.32,74200.0,5610.0,46226.44,145703.25
DECK,19284.80,69286.0,35300.0,61263.53,75827.85
RS,53259.58,108144.0,314480.0,80783.79,48266.90
CSL,69556.75,94979.0,0.0,81715.90,108476.74
GDDY,173792.23,29718.0,35370.0,71148.20,139208.91
VST,140103.49,40506.0,65110.0,70439.04,243464.27
GGG,147860.53,81009.0,65080.0,95593.55,-116915.10
MANH,-58436.59,71866.0,171870.0,61721.84,13483.13
LII,204278.26,68599.0,0.0,85548.97,126550.93
WSM,-40857.16,52530.0,64960.0,54772.95,90535.65


# Cumulative Return Calculation:

In [192]:
df_c = pd.read_csv('/content/drive/My Drive/Portfolio_Optimization/latest_pctg.csv', index_col='Date')

In [193]:
df_c.head()

,SMCI,DECK,RS,CSL,GDDY,VST,GGG,MANH,LII,WSM,SAIA,LECO,RPM,XPO,^SP400
Date,,,,,,,,,,,,,,,
2023-10-03,-0.065395,-0.018220,-0.018500,-0.002383,-0.005264,-0.016293,-0.004938,-0.018757,-0.014803,-0.022891,0.007855,0.004248,-0.014757,0.000817,-0.016193
2023-10-04,0.038634,0.005432,0.009522,0.018437,0.000814,-0.019063,0.025086,0.021575,0.024767,0.006437,0.004517,0.006234,0.062608,0.011022,0.005193
2023-10-05,0.025678,-0.028050,0.003183,0.009267,-0.003932,0.014017,-0.011967,0.000853,-0.000967,-0.003627,-0.016826,-0.000774,0.000710,0.002288,-0.002089
2023-10-06,0.007198,-0.002960,0.009054,0.016271,0.012386,-0.009111,0.018100,0.036790,0.004301,0.028656,0.015254,0.020539,-0.005776,0.035585,0.008034
2023-10-09,0.001139,0.008988,0.006519,0.007129,0.001613,0.009829,0.005748,0.006575,0.016354,0.030110,0.014505,0.002061,-0.002650,0.018024,0.007559


In [194]:
Cumulative = (1+df_c).cumprod() - 1

In [195]:
Cumulative.shape

(102, 15)

In [196]:
# Matrics Multiplication:
x = PF_Value.values.T
y = Cumulative.values
model = PF_Value.columns.to_list()
stk_name = Cumulative.columns.to_list()

dfs = {} # Dictionary for store multiple dataframe

for i in range(0,len(model)):

  result = x[i,:]*y
  dfs[f'df_{model[i]}'] = pd.DataFrame(result, columns=stk_name, index=df_c.index)

# column sum function:
calculate_total = lambda row: row.sum()

for key, df in dfs.items():
    df['total'] = df.apply(calculate_total, axis=1)



In [197]:
# Assign dataframe from dictionary dataset of above declaration:
df_MVO = dfs['df_MVO']
df_TB = dfs['df_TB']
df_BL = dfs['df_BL']
df_SRM = dfs['df_SRM']
df_MDP = dfs['df_MDP']

In [198]:
import plotly.graph_objects as go
fig = go.Figure()
i=0
for key, df in dfs.items():
    total_column = df['total']
    key = model[i]
    fig.add_trace(go.Scatter(x=df.index, y=total_column, mode='lines',name=key))
    i=i+1
fig.update_layout(title='Model Analysis', xaxis_title='Date', yaxis_title='Returns')

fig.update_xaxes(
    tickvals=pd.date_range(start=df.index.min(), end=df.index.max(), freq='W-MON'),
    tickformat='%Y-%m-%d',
    tickangle=-45
)
fig.show()
